***
# LITERATURE
***

***
# Deep reinforcement learning for turbulent drag reduction in channel flows
***
> Guastoni, L., Rabault, J., Schlatter, P., Azizpour, H., & Vinuesa, R. (2023). Deep reinforcement learning for turbulent drag reduction in channel flows. The European Physical Journal. E, Soft Matter, 46.
> 
> Latex :
> > @article{Guastoni2023DeepRL,
  title={Deep reinforcement learning for turbulent drag reduction in channel flows},
  author={Luca Guastoni and Jean Rabault and Philipp Schlatter and Hossein Azizpour and Ricardo Vinuesa},
  journal={The European Physical Journal. E, Soft Matter},
  year={2023},
  volume={46},
  url={https://api.semanticscholar.org/CorpusID:256194495}
}

## Goal 
Enhance drag reduction with flow control using RL.

## How 
$$\text{DRL with MARL}$$

![Figure 1 du papier](Images/Literature/MARL_DRL_Lucas.png)

- **Control Policy and Evaluation:**
  - Each agent computes wall-normal velocity intensity to be applied.
  - Control policy evaluated over fixed time interval.
  - Actual control varies linearly from old value to new one to avoid numerical instabilities.
  - Time units scaled with $t^* = \frac{\nu}{u_{\tau}^2}$, where $\Delta t^+ = 0.6$ in numerical experiments.
  - Actuation value limited to range $[-u_{\tau}, u_{\tau}]$.
 
- **Observation for Agent:**
  - Agent observes portion of one or more wall-parallel planes of sampled flow quantities.
  - Observable portion of flow field above actuation.
  - Sampling heights referred using inner-scaled wall-normal coordinates: $y^+ = \frac{y u_{\tau}}{\nu} = \frac{y}{l^*} \in [0, Re_{\tau}]$, where $l^*$ is viscous length.
 
- **Deep Reinforcement Learning Algorithm (DDPG):**
  - Optimizes deep neural network (actor) approximating relation between state and action (policy function).
  - Policy deterministic after learning.
  - Second neural network (critic) approximates action-value function $Q(s, a)$.
  - Simple fully connected networks used for both actor and critic.
    - Actor: Single layer of 8 nodes.
    - Critic: Three layers with 16, 64, and 64 nodes.
  - Gaussian noise with zero mean and variance $0.1u_{\tau}$ added to actions computed by actor network for exploration.
  - Policy updated every $\Delta t^+ = 180$, with 64 mini-batch gradient updates.
  - Mini-batches sampled from buffer replay including 5,000,000 (state, action, reward) tuples.
  - Default parameter values from Stable-baselines implementation used.
  - Configuration details available in associated code repository.

### Simualtion Setup 

- **Simulation Domains:** 
  - **Minimal Channel:** Ω = $2.67h \times h \times 0.8h$
  - **Larger Channel:** Ω = $2\pi h \times h \times \pi h$


- **Velocity Components:** 
  - $(x, y, z)$ represent streamwise, wall-normal, and spanwise directions respectively.
  - $(u, v, w)$ represent corresponding velocity components.


- **Friction Reynolds Number:** 
  - $Re_{\tau} = \frac{u_{\tau}h}{\nu}$
  - $u_{\tau} = \frac{\tau_{w}}{\rho}$
  - $\tau_{w}$ is the wall-shear stress, $\rho$ is fluid density, and $\nu$ is kinematic viscosity.


- **Reynolds Number:** 
  - $Re_{\tau} = 180$ in both domains.

- **Solver:** SIMSON (DNS coded in FORTRAN 77/90)
  - Pseudo-spectral code using Chebyshev polynomials in wall-normal direction.


- **Resolution:**
  - **Minimal Channel:** $16 \times 65 \times 16$
  - **Larger Domain:** $64 \times 65 \times 64$


- **Time-Advancement Scheme:** 
  - Second-order Crank-Nicholson for linear terms.
  - Third-order Runge-Kutta for nonlinear terms.



How to deal with the solver-DRL Interface

- **Reinforcement Learning Framework and Environment:**
  - Framework: Stable-baselines 
  - Environment: Custom PettingZoo/Gym environment
  - Both written in Python.
  - Fluid solver coded in FORTRAN 77/90.

- **Interface Between Programming Languages:**
  - Interface based on message-passing interface (MPI) in this case.
  - Fluid solver spawned as child process of main learning process.
  - Intercom created between solver and Python main program.
  - Communication requests handled using five-character strings:
    - **STATE:** Request simulation to communicate state to agent.
    - **CNTRL:** Agent communicates new action to environment, updating controllable parameters in fluid solver.
    - **EVOLVE:** Compute several time iterations of solver after CNTRL update to observe effect of chosen action on environment. Instantaneous wall-shear stress passed to Python program for reward computation.
    - **TERMN:** Interrupt solver and close FORTRAN program. Used at end of each episode before restarting environment. Initial condition same for every episode unless stated otherwise.

- **Functions for Gym Environment API:**
  - Combination of messages used to define required functions.


## Validation and Baseline

- **Opposition Control as Baseline:**
  - Established control benchmark for assessment of DRL-control laws.
  - Actuation at wall $v_{\text{wall}}$ by opposition control computed using expression:
    $v_{\text{wall}}(x,z,t) = -\alpha v(x,y_s,z,t) - \langle v(x,y_s,z,t) \rangle$
  - $\langle v(x,y_s,z,t) \rangle$: Spatial mean of wall-normal velocity field.
  - Second term enforces zero-net-mass-flux within domain.
  - $\alpha$: Positive scaling parameter fixed in spatial directions and time (typical value $\alpha = 1$).

- **Validation of Code Implementation:**
  - Reproduced opposition control strategy in solver.
  - Environment evolved with constant $\alpha$ until drag-reduction rate reached stationary value.
  - Previous studies reported highest drag reduction at $y^+ = 10$.
  - Computational domain size: $\Omega = L_x \times L_y \times L_z = 4\pi h \times h \times \frac{4}{3}\pi h$, resolution $N_x \times N_y \times N_z = 128 \times 129 \times 128$.
  - Friction Reynolds number $Re_{\tau} = 180$, Reynolds number based on centreline velocity $Re_{cl} = 3273$.
  - Drag reduction in literature: $\approx 14\%$ at $y^+ = 10$.
  - Their result in larger channel: $17.73\%$, acceptable agreement considering lower resolution and small difference in spanwise size of domain.

- **Validation with Different Sampling Planes:**
  - Ref. [50]: Sampling at $y^+ = 15$, $Re_{cl} = 3240$.
    - Drag reduction: $\approx 25.67\%$, very good agreement.
  - Ref. [52]: Sampling at $y^+ = 15$, $Re_{\tau} = 180$.
    - Drag reduction: $\approx 24\%$, good agreement with our result.
  
- **Overall Validation and Baseline:**
  - Results match those reported in literature, validating simulation setup and control implementation.
  - Drag reduction obtained with opposition control used as baseline for comparison with DRL results.


## Results

- **Traditional Control Strategy:**
  - Achieves approximately 20% drag reduction.

- **Current Approach:**
  - **Minimal Channel:** Achieves approximately 40% drag reduction.
  - **Larger Channel:** Achieves approximately 30% drag reduction.

### Considerations:
- The RL strategy incurs a slightly higher computational cost, although negligible in comparison to the achieved drag reduction.

- Investigation into the effects of varying Reynolds number (Re) on the obtained results is warranted.

## How to Utilize this Paper for Our Needs?

- **Baseline Comparison:**
  - Use the results presented in this paper as a baseline for evaluating our own codes.
  - Ensure our codes are tested with exactly the same parameters as described in this paper.
  - **Reference to outperfom**.

This approach allows for a direct comparison of our results with those reported in the paper, providing a valuable benchmark for assessing the performance and accuracy of our codes.

*** 
# Identifying regions of importance in wall-bounded turbulence through explainable deep learning
***


> Cremades, A., Hoyas, S., Deshpande, R., Quintero, P., Lellep, M., Lee, W.J., Monty, J.P., Hutchins, N., Linkmann, M., Marusic, I., & Vinuesa, R. (2023). Identifying regions of importance in wall-bounded turbulence through explainable deep learning.
> 
> Latex :
> > @inproceedings{Cremades2023IdentifyingRO,
  title={Identifying regions of importance in wall-bounded turbulence through explainable deep learning},
  author={Andres Cremades and Sergio Hoyas and Rahul Deshpande and Pedro Quintero and Martin Lellep and Will Junghoon Lee and Jason P. Monty and Nicholas Hutchins and Moritz Linkmann and Ivan Marusic and Ricardo Vinuesa},
  year={2023},
  url={https://api.semanticscholar.org/CorpusID:256504028}
}

## Importants theoretical notions in the article

### What is a coherrent structure ? 

A coherent structure in fluid dynamics refers to a specific pattern or organization of fluid motion that persists over time, despite the inherently chaotic nature of fluid flow. These structures are identifiable entities within the flow that exhibit some degree of organization or coherence.

**Definition:**
- A coherent structure is a specific pattern or organization of fluid motion that persists over time despite the inherent chaotic nature of fluid flow.

**Examples:**
- Vortices (tourbillons): Swirling regions of fluid motion characterized by a rotational flow pattern.
- Eddies (remous): Whirling motions of fluid, typically found downstream of obstacles or in turbulent flows.
- Jets (jets): High-velocity streams of fluid ejected into a surrounding medium.
- Wakes (sillages): Regions of disturbed flow downstream of solid objects or obstacles.
- Boundary layers (couche limites): Layers of fluid adjacent to a solid boundary exhibiting distinct flow characteristics.

**Literature Examples:**
- Streamwise streaks: Regions within a fluid flow where the velocity of fluid particles align predominantly in the direction of the flow, known as the streamwise direction. These streaks appear as elongated regions of higher or lower velocity compared to the surrounding fluid. [3]
- Reynolds-stress events: Fluctuations in the stress tensor components caused by turbulent fluctuations in velocity in a fluid flow. [22]
- Q-event (object of this paper): [This would need a specific definition or reference to explain further.]

Coherent structures play a crucial role in understanding and analyzing fluid flows, providing insights into the dynamics and behavior of complex flow phenomena.

#### Extra explanation from Marco thesis p29-30

> Atzori, M. (2021). 
(Doctoral dissertation, KTH Royal Institute of Technology).
> > 
> Latex :
> > @phdthesis{atzori2021coherent,
  title={Coherent structures and control in wall-bounded turbulent flows},
  author={Atzori, Marco},
  year={2021},
  school={KTH Royal Institute of Technology}
}

- **Coherent Structures in Turbulent Flows:**
  - Incompressible turbulent flows share common governing equations and general properties.
  - Distinct phenomena arise due to geometrical constraints and symmetry properties in different flows.
  - Intense Reynolds-stress events in square/rectangular ducts and APG turbulent boundary layers were studied.
  - **Definition of Intense Reynolds-Stress Events:**
    - Connected regions with instantaneous velocity fluctuations higher than a threshold.
    - Based on root-mean-square of velocity components.
    - Extension of classic quadrant analysis by Lozano-Durán et al. (2012).
  - **Threshold Conditions:**
    - Criteria for off-diagonal components of Reynolds-stress tensor: 
      $ |uv| > H_{uv}u_{rms}v_{rms}, \, |uw| > H_{uw}u_{rms}w_{rms}, \, |vw| > H_{vw}v_{rms}w_{rms} $
    - Scalar thresholds $ H_{XX} $ ("hyperbolic hole") with normalization terms $ u_{rms}v_{rms} $, $ u_{rms}w_{rms} $, $ v_{rms}w_{rms} $.
    - Chosen for homogeneous detection of coherent structures.
  - **Percolation Analysis:**
    - Introduced by Moisy & Jiménez (2004) and used by Lozano-Durán et al. (2012).
    - Evaluates effectiveness of threshold with the ratio: 
      $ \frac{V_{max}}{V_{all}} $
    - Indicates transition between homogeneous and non-homogeneous flow.
  - **Percolation Crisis:**
    - Common in systems with high degrees of freedom and specific connectivity/statistical properties.
    - Used in fluid mechanics beyond structure identification, e.g., laminar-to-turbulent transition.
  - **Statistical Properties and Percolation Crisis:**
    - Paper 12 discusses the relationship between statistical properties of a scalar field and percolation crisis.


### What is Reynolds Stress ?

Reynolds stress ($\tau_{ij}$) is a fundamental concept in fluid mechanics, particularly in the study of turbulent flows. It represents the statistical average of the fluctuating component of the momentum transport caused by turbulent velocity fluctuations within a fluid flow. 

**Definition:**
Reynolds stress is the average stress caused by fluctuations in velocity in a turbulent flow. It arises due to the interaction between turbulent eddies of different sizes, leading to variations in momentum transfer within the flow.

**Reynolds Decomposition:**
The Reynolds decomposition separates the velocity field into its mean ($\overline{u_i}$) and fluctuating ($u_i'$) components:
$$
u_i = \overline{u_i} + u_i'
$$

**Hypothesis for Writing Reynolds Stress:**
- The Reynolds decomposition assumes that the flow variables can be decomposed into a mean part and a fluctuating part.
- The Reynolds stress tensor $\tau_{ij}$ is defined as the average of the product of the fluctuating velocities $u_i'$ and $u_j'$:
$$
\tau_{ij} = \overline{u_i' u_j'}
$$

**Components:**
Reynolds stress has both normal and shear stress components, representing momentum transfer in different directions within the flow. 
- The normal stress components represent momentum transfer perpendicular to the flow direction.
- The shear stress components represent momentum transfer parallel to the flow direction.

**Importance:**
Reynolds stress is crucial for understanding turbulence dynamics and predicting turbulent flow behavior in various engineering and scientific applications. It influences the overall momentum transfer, heat transfer, and mixing processes within turbulent flows.

**Measurement:**
Reynolds stress can be measured experimentally using various techniques such as hot-wire anemometry, particle image velocimetry (PIV), and laser Doppler velocimetry (LDV). These experimental methods allow researchers to quantify the magnitude and distribution of Reynolds stress in turbulent flows.

**Modeling:**
In computational fluid dynamics (CFD) simulations, Reynolds stress is often modeled using turbulence models such as the Reynolds-averaged Navier-Stokes (RANS) equations or large eddy simulation (LES) models. These models aim to predict the behavior of turbulent flows by simulating the statistical characteristics of Reynolds stress.


### What are Q-Events ?

Based on reference [22], it serves as the primary energy source for all structures within turbulent flow. Hence, its study holds significant intrigue and importance.

**Definition**: Q-events denote distinct flow regions intricately linked with momentum transfer and the generation of turbulent kinetic energy (TKE), invariably contributing to substantial drag within the system.

**Important properties**: 
- These are delineated as coherent zones exhibiting instantaneous high Reynolds stress.
- Their identification hinges on the condition:
$$u(x, y, z, t)v(x, y, z, t) > Hu'(y)v'(y),$$
where $H$ represents a hyperbolic function. (This criterion serves as a crucial tool for detection.)
- Q-events effectively highlight regions characterized by statistically significant magnitudes of Reynolds stress.
- Quadrant analysis categorizes them into four distinct types:
  1. **Outward Interactions:** Where $u' > 0$ and $v' > 0,$
  2. **Ejections:** Featuring $u' < 0$ and $v' > 0,$
  3. **Inward Interactions:** With $u' < 0$ and $v' < 0$,
  4. **Sweeps:** Characterized by $u' > 0$ and $v' < 0$.

Understanding these coherent structures not only provides insights into the dynamics of turbulent flows but also facilitates the development of strategies for managing associated drag and optimizing various processes.

#### Extra explanation from Luca thesis p32-33

> Guastoni, L. (2023). Time, space and control: deep-learning applications to turbulent flows (Doctoral dissertation, KTH Royal Institute of Technology).
> > 
> Latex :
> > @phdthesis{guastoni2023time,
  title={Time, space and control: deep-learning applications to turbulent flows},
  author={Guastoni, Luca},
  year={2023},
  school={KTH Royal Institute of Technology}
}

<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/quadrant-analysis.png" alt="Figure 1 du papier" width="600">
</div>


The $(v',u')$ plot is called the quadrant analysis.

- Quadrant analysis is a tool used to analyze controlled flow in various strategies.
- Introduced by Wallace et al. (1972) to categorize contributions to Reynolds shear stresses.
- Divides streamwise and wall-normal fluctuations into four categories: Q1, Q2, Q3, Q4.
- Q2 and Q4 events correspond to ejections and sweeps in turbulent flows.
- Predominance of Q4 near the wall and Q2 farther from the wall in wall-bounded turbulence.
- Q1 and Q3 interactions are outward and inward, respectively, and less frequent.
- Q2 and Q4 motions are gradient-type interactions related to vertical momentum fluxes.
- Q1 and Q3 are counter-gradient-type interactions.
- Effective in revealing the connection between fluctuation distribution and turbulent features.
- Used to identify how different control strategies affect flow physics.



### About the Paper:

<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/Andres_Workflow.png" alt="Figure 1 du papier" width="400">
</div>



**Objective:** The aim is to utilize U-nets for predicting the evolution of turbulent channel flow.

**Data:** The study is based on 6,000 instantaneous realizations obtained from simulations of turbulent channel flow.

**Workflow:**
- **Domain Segmentation:** The domain is segmented into Q events.
- **Feature Extraction:** Each structure is considered as an input feature for the SHAP algorithm.
- **SHAP Algorithm:** Game-theoric method that calculates the importance of each feature on the U-net prediciton.

**Process:**
- **Flow Prediction:** Utilization of a U-net for flow prediction.
- **Structure Evolution:** Advancing a time step in the simulation to determine structure evolution.
- **SHAP Values:** Quantification of the importance of each coherent turbulent structure using SHAP values.
- **Comparison:** Comparison of the predicted solution with the simulated flow field in the next time step.

**Analysis:**
- Identification of coherent structures representing ejections and sweeps.
- Utilization of Explainable AI (XAI) to define an objective metric, confirming and extending previous results.

**Previous Approaches:**
- **Encinar and Jiménez [33]:** Studied homogeneous isotropic turbulence by introducing perturbations and numerically integrating governing equations.
- **Lozano-Durán and collaborators [34, 35]:** Employed information theory to identify regions of flow with the highest impact on future flow states.

**Advantages of Present Approach:**
- Reliance on deep learning provides a good surrogate for governing equations, even in environments where full access to DNS data is not possible, such as experiments.
- Allows assessment of important flow regions at higher Reynolds numbers and with less available data.

**Importance:** The significance of this approach is demonstrated in its applications in flow prediction and flow control.

**Simulation parameters:**
- $Re_{\tau} = 125$,
- Poiseuille flow : **Channel** dimensions Lx = 2πh, Lz = πh, with streamwise periodicity. (h ? ,y ?)


***
# More about coherent structures from Marco paper 
***

# Geometry and clustering of intense structures in isotropic turbulence

"(Moisy & Jiménez 2004) introduced the percolation analysis in their study of vortical and dissipative coherent structures in homogeneous isotropic turbulence", Marco.

> MOISY, F., & JIMÉNEZ, J. (2004). Geometry and clustering of intense structures in isotropic turbulence. Journal of Fluid Mechanics, 513, 111–133. doi:10.1017/S0022112004009802
> 
> Latex :
> > @article{MOISY_JIMÉNEZ_2004, title={Geometry and clustering of intense structures in isotropic turbulence}, volume={513}, DOI={10.1017/S0022112004009802}, journal={Journal of Fluid Mechanics}, author={MOISY, F. and JIMÉNEZ, J.}, year={2004}, pages={111–133}} <div></div>


#### What I want to learn from this paper 
What is a percolation analysis ?

## Quick reading 

*From 1.Introduction*
### Main Points:
- The paper addresses the interplay between geometric aspects and statistical properties of intense vortical structures in turbulent flows.
- It discusses the difficulty in characterizing intermittency in turbulent flows, either from the geometry of structures or from statistical fluctuations at different scales.
- The paper highlights the challenges in connecting statistical quantities with the geometry of individual structures and their spatial distribution.
- Various criteria and methods for identifying intense vortical structures are discussed, including pressure criteria, eigenvalues of the velocity gradient tensor, and fractal analysis.
- Experimental and numerical evidence suggests non-random spatial distributions and clustering of intense vortical structures.
- The paper aims to investigate the links between fractal and structural descriptions of turbulence.

### Variables and Definitions:
1. **Vorticity Magnitude ($\omega$)**: Measure of the local rotation of fluid particles in a flow.
2. **Rate-of-Strain Tensor ($S_{ij}$)**: Describes the stretching and deformation of fluid elements in a flow.
3. **Pressure ($p$)**: Represents the scalar field associated with the force exerted by a fluid.
4. **Eigenvalues of the Velocity Gradient Tensor ($\lambda_1, \lambda_2, \lambda_3$)**: Represent the magnitudes of the stretching and rotation of fluid elements in different directions.
5. **Fractal Dimension ($D$)**: Measure of the complexity of a geometric structure in terms of its scaling properties.
6. **Multifractal Exponents**: Describe the scaling behavior of different parts of a multifractal structure.

### Formulas:
1. **Pressure Poisson Equation**: $\nabla^2 p / \rho = Q = 1/2 \omega^2 - S_{ij}S_{ij}$
   - Explanation: Represents the balance between vorticity and strain rate in determining pressure distributions.
2. **Fractal Dimension Formula**: $D \approx 2.7$
   - Explanation: Provides a measure of the complexity of the spatial distribution of intense vortical structures.


*From 2.Data Generation*

### Main Points:
- The paper analyzes numerical simulations conducted by Jiménez et al. (1993) to study turbulent flows.
- Data consists of two instantaneous velocity fields of three-dimensional forced turbulence with 512^3 collocation points and periodic boundary conditions.
- Resolution in physical space is achieved with a scale separation of approximately 200 (L0/η).
- Quantities of interest include vorticity magnitude and strain-rate magnitude.
- Statistical and geometrical properties of sets defined by thresholding vorticity and strain rate fields are studied.

### Variables and Definitions:
1. **Vorticity Magnitude ($|\omega|$)**: Measure of the local rotation of fluid particles in a flow.
2. **Strain-Rate Magnitude ($|s|$)**: Describes the stretching and deformation of fluid elements in a flow.
3. **Energy Dissipation Rate ($\varepsilon(x)$)**: Rate at which energy is dissipated in the flow, calculated from the strain-rate magnitude.
4. **Kinematic Viscosity ($\nu$)**: Measure of fluid's resistance to deformation.
5. **Root-Mean-Square Fluctuations ($\omega'$ and $s'$)**: Represent the fluctuations of vorticity and strain rate, respectively.
6. **Mean Dissipation Rate ($\langle \varepsilon \rangle$)**: Average dissipation rate of energy in the flow.

### Formulas:
1. **Local Energy Dissipation Rate**: $\varepsilon(x) = 2\nu|s(x)|^2$
   - Explanation: Computes the rate of energy dissipation at a specific point in the flow based on the strain-rate magnitude.
2. **Vorticity and Strain-Rate Balance**: $\omega'^2 = 2s'^2 = \langle \varepsilon \rangle / \nu$
   - Explanation: Represents the balance between vorticity and strain rate on average in the flow.



*From 6.Concluding remarks*

**Main Points:**
- Box-counting methods applied to high vorticity and strain rate sets in isotropic turbulence at Reλ ≃ 168.
- No clear fractal behavior observed due to limited scale range.
- Minimum of logarithmic slope of box counts provides upper bound for possible fractal dimension.
- Local exponents provide estimates for fractal dimension of vorticity and dissipation field.
- Intense sets split into disconnected structures for high thresholds (< 4% volume fraction).
- Fractal properties influenced by structure shape, size distribution, and arrangement of baricenters.
- Strain-rate structures resemble flat sheets or ribbons, vorticity structures vary from ribbons to tubes.
- Intense structures not uniformly distributed, but clustered into inertial size clusters.
- Potential for scale-independent nesting of intense structures at higher Reynolds numbers.
- Decrease in global dimension with increased threshold affects one-point measurements.
- Dimensions for highest thresholds decrease to low values (~0.2), suggesting possible isolated objects.
- Different behavior observed for dynamical and passive fields.

**Variables and Definitions:**
- Reλ: Reynolds number.
- Dα∗: Minimum of logarithmic slope of box counts, upper bound for fractal dimension.
- D∗: Local exponents providing estimates for fractal dimension.
- |ω|, ω′, |s|, s′: Magnitude and fluctuations of vorticity and strain rate.
- Dω, Ds: Individual dimensions for vorticity and strain-rate structures.
- Aspect ratios: Ratio of structure dimensions.
- Baricenters: Centers of mass of structures.
- τ: Threshold for one-point measurements.
- δr: Length of free intervals in sets.

**Formulas:**
1. $D_{\alpha^*}$: Minimum of logarithmic slope of box counts.
2. $D^*$: Local exponents providing estimates for fractal dimension.
3. $D_{\omega}$, $D_{s}$: Individual dimensions for vorticity and strain-rate structures.
4. $\tau$: Threshold for one-point measurements.
5. $\delta r$: Length of free intervals in sets.

# The three-dimensional structure of momentum transfer in turbulent channels

"and Lozano-Dur ́an et al. (2012) employed it to qualitatively assess the effectiveness of the threshold.", p.39, Marco.

> Lozano-Durán, A., Flores, O., & Jiménez, J. (2012). The three-dimensional structure of momentum transfer in turbulent channels. Journal of Fluid Mechanics, 694, 100–130. doi:10.1017/jfm.2011.524
> 
> Latex :
> > @article{Lozano-Durán_Flores_Jiménez_2012, title={The three-dimensional structure of momentum transfer in turbulent channels}, volume={694}, DOI={10.1017/jfm.2011.524}, journal={Journal of Fluid Mechanics}, author={Lozano-Durán, Adrián and Flores, Oscar and Jiménez, Javier}, year={2012}, pages={100–130}} <div></div>


#### What I want to learn from this paper 
What is a percolation analysis ?



# Quick reading 

## History of the statistical properties of the Reynolds stresses

### Beginning p. 101
The study of the statistical properties of the Reynolds stresses began in the late 1960s, with the observation by Kline et al. (1967) that the near-wall low-velocity streaks undergo a process of lift-up, oscillation, break-up and ejection, which they called bursting. Kim, Kline & Reynolds (1971) showed that most of the turbulence production in the near-wall region occurs during those bursts, and several conditional- sampling techniques were developed to identify the structures involved in the process. Examples are the u-level detection of Lu & Willmarth (1973), the VITA (variable- interval time average) of Blackwelder & Kaplan (1976) and the VISA (variable- interval space average) of Kim (1985). Several of those techniques were surveyed by Bogard & Tiederman (1986), who concluded that the best compromise between detection probability and false positives was provided by the quadrant analysis of Wallace, Eckelman & Brodkey (1972), Willmarth & Lu (1972) and Lu & Willmarth (1973), in which points of the flow are classified in terms of the quadrant of the parameter plane of streamwise and wall-normal velocity fluctuations. Q1 events (outward interactions) have u > 0 and v > 0, Q2 events (ejections) have u < 0 and v > 0, Q3 events (inward interactions) have u < 0 and v < 0 and Q4 events (sweeps) have u > 0 and v < 0. Hereafter, Q events will be referred to simply as Qs, and we will occasionally group the ‘gradient’ Q2s and Q4s as Q−, and the ‘counter-gradient’ Q1s and Q3s as Q+.

### After p. 101

Although most of the early work focused on bursting in the buffer region, studies during the 1990s began to establish a relationship between near-wall bursting and outer structures. Antonia, Bisset & Browne (1990) applied a variant of the u-level method (WAG, window average gradient) to boundary layers with relatively high Reynolds numbers based on the momentum thickness, Reθ = U∞θ/ν = 1360–9630, corresponding to h+ = 530–3100. They reported that outer-scale discontinuities in the streamwise velocity, extending across the whole thickness of the boundary layer, tend to be located near the upstream end of the near-wall bursts. Wark & Nagib (1991) applied quadrant analysis to a boundary layer at Reθ = 4650 (h+ = 1500), and found large conditional events with sizes of the order of h. Using a related identification technique in the atmospheric boundary layer, Narasimha et al. (2007) described Reynolds-stress structures with lengths comparable to the distance to the wall that, because of the high Reynolds number involved, were both very long in wall units and very short compared with the boundary layer thickness. Hoyas & Jime ́nez (2006) showed that the spectral signatures of the very-large motions (Jime ́nez 1998; Kim&Adrian1999),orglobalmodes(delA ́lamoetal.2004),characteristicofthe outer region are present at the wall, as well as those of similar smaller structures in the logarithmic layer, and Hutchins & Marusic (2007b) showed that part of that influence is a modulation of the intensity of the small-scale buffer-layer fluctuations.

### Recntly p. 102


More recently, Flores & Jime ́nez (2010a) showed that the hierarchy of structures of increasing sizes postulated by Townsend (1961) for the logarithmic layer can be represented numerically in computational boxes of widths proportional to the height to be studied. Each box contains a single complex structure, including a segment of a streamwise velocity streak and a cluster of vortices, that bursts quasiperiodically to fill the box with essentially normal turbulence, with a strong Q2 and Q4 occurring simultaneously. The existence of a hierarchy of Q2s and Q4s was also conjectured by Wark & Nagib (1991), and is consistent with the similarity observed by Kailas & Narasimha (1994) between the conditionally sampled structures extracted from the near-wall region of laboratory boundary layers and those in the logarithmic region of the atmospheric boundary layer.
Q2s and Q4s play important roles in most of the structural models proposed to explain how turbulent kinetic energy and momentum are redistributed in wall-bounded turbulence. The majority of these models (see the review by Robinson 1991) are loosely based on the attached-eddy hypothesis of Townsend (1961) and involve wall- attached vortical loops growing from the wall into the outer region (Perry, Henbest & Chong 1986). More recently, in order to account for the very large-scale motions mentioned above, as well as for earlier experimental evidence on the internal structure of the bursts (Bogard & Tiederman 1986), a variant model has evolved to include vortex packets (Adrian, Meinhart & Tomkins 2000), in which trains of hairpin vortices grow from the wall by a process of merging (Tomkins & Adrian 2003) and self- regeneration (Zhou et al. 1999). The heads of the hairpins are arranged into inclined shear layers generated by the combined induction of the hairpins of the packet, with a Q4 upstream and above the packet and a Q2 beneath it. Ganapathisubramani, Longmire & Marusic (2003) used PIV data on wall-parallel planes in a boundary layer with Reθ = 2500 (h+ = 1060) to quantify the contribution to the Reynolds stresses by packets of hairpins. They report that, at y+ = 100, packets covering 4 % of the area produce 28 % of the total Reynolds stresses.
The hairpin-packet model is usually understood to involve relatively smooth vortex loops as the structures associated with Townsend’s (1976) attached eddies, although recent discussions tend to include ‘canes, heads, legs and three-quarter hairpin shapes, generally asymmetric and distorted’ into the class of ‘hairpin-like’ vortices (Dennis & Nickels 2011a), making them hard to distinguish from generic vortices. An alternative,lessorganized,structurewasproposedbydelA ́lamoetal.(2006),who analysed the statistical properties of clusters of vortices (clusters hereafter) in direct numerical simulations (DNSs) of turbulent channels with h+ = 550–1880. The clusters segregate into wall-attached and wall-detached families. The wall-detached clusters are dissipative objects, with sizes that scale with the local Kolmogorov scale. The attached ones form a self-similar family of objects that mark strong Q2s. The averaged flow conditioned to an attached cluster consist on an elongated low-velocity streak extending downstream of the cluster, flanked by a pair of shorter high-velocity streaks. While this averaged structure is consistent with the hairpin packet paradigm, del A ́lamoetal.(2006)pointedoutthattheinstantaneousclustersaremorecomplex,in agreement with earlier visualizations by Tanahashi et al. (2004).

#### What is an Extractor ?
An exctractor is an other word for describing the Q-events.

#### What is Percolation Analysis?

Percolation analysis involves ploting the detection rate of extractors as a function of the parameter $H$, with the aim of maximizing this rate as in the following figure.


### About $H$ and percolation diagram p. 105

<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/percolation_diagram.png" alt="Figure 1.a; du papier" width="700">
</div>

Note that the threshold in (2.1) depends on the wall distance. As noted by Nagasoa & Handler (2003), using an identification method with a constant threshold, such as |τ| > Hu′(y0)v′(y0), with some reference y0, is problematic in inhomogeneous flows. In our case, very few objects are recovered in the outer region if y0 is chosen in the buffer layer, while choosing a high y0 results in a confusingly cluttered buffer layer. This behaviour agrees with the observations of Blackburn, Mansour & Cantwell (1996) and worsens as Re increases. Our choice of the local standard deviation in (2.1) results in a roughly constant ́contribution of the Qs to the Reynolds stress and agrees with a similar choice in del Alamo et al. (2006) for the identification of vortex clusters.
As argued by Bogard & Tiederman (1986), quadrant analysis depends on the value of the hyperbolic-hole size H. They report that the optimum threshold for the buffer region is H ≈ 1, based on direct comparisons between the detected events and instantaneous visualizations of the flow. We choose H based on the percolation behaviour of (2.1). Percolation theory describes the statistics of the connected components of a random graph. Here, it is applied to the variation with H of the volume of the connected objects extracted by (2.1), as first used to identify vorticity and dissipation structures in isotropic turbulence by Moisy & Jime ́nez (2004) and in channelsbydelA ́lamoetal.(2006).

Figure 1(a) shows the percolation diagram of (2.1) in the two channels considered here. The solid lines are the ratio of the volume of the largest identified object, Vlar, to the total volume Vtot satisfying (2.1), and the dashed ones are the total number of identified objects, N/Nmax , normalized with its maximum over H. When H & 3, the identification only yields a few small objects that correspond to the strongest Qs. Decreasing H introduces new Qs, while previously identified ones grow in size. At first, the size of the largest Q changes little, and Vlar/Vtot stays roughly constant, but eventually the objects start to merge, resulting in a rapid increase of Vlar /Vtot and in a decrease of the number of objects. Figure 1(a) shows that this percolation crisis takes place in the range 0.5 . H . 3, independently of the Reynolds number. For lower thresholds, Vlar /Vtot ≈ 1, and most of the volume satisfying (2.1) belongs to a single object.

***
# The minimal flow unit in near-wall turbulence
***
> Jiménez, J., & Moin, P. (1991). The minimal flow unit in near-wall turbulence. Journal of Fluid Mechanics, 225, 213–240. doi:10.1017/S0022112091002033
> 
> Latex :
> > @article{Jiménez_Moin_1991, title={The minimal flow unit in near-wall turbulence}, volume={225}, DOI={10.1017/S0022112091002033}, journal={Journal of Fluid Mechanics}, author={Jiménez, Javier and Moin, Parviz}, year={1991}, pages={213–240}} <div></div>

#### Why study this paper ?

This paper is crucial because it serves as the origin of the minimal channel flow case, establishing it as the canonical state. Therefore, it is essential to thoroughly examine the setup provided in the paper.


<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/minimal_channel_domain.png" alt="Figure 1 du papier" width="700">
</div>

## 1. The Physics of the Problem

### Geometry
The computational domain is a rectangular box with physical no-slip walls at the top and bottom and periodic boundaries in both the streamwise ($x$) and spanwise ($z$) directions. The flow is considered to be fully developed channel flow.

### Symmetries and Invariance
The problem exhibits translational symmetry along the $x$ and $z$ directions due to the periodic boundary conditions. The flow is assumed to be symmetric across the channel's centerline in the absence of system-wide gradients or external forces that would break this symmetry.

### Equations Simulated
The Navier-Stokes equations for incompressible flow are simulated:

$$
\frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} = -\nabla p + \nu \nabla^2 \mathbf{u},
$$

$$
\nabla \cdot \mathbf{u} = 0,
$$

where $\mathbf{u}$ is the velocity field, $p$ is the pressure divided by the constant density (assumed incompressible), and $\nu$ is the kinematic viscosity.

### Boundary Conditions
- **Top and Bottom Walls:** No-slip condition ($\mathbf{u} = 0$).
- **Streamwise and Spanwise Directions:** Periodic.

### Initial Conditions and Reynolds Number
The flow is initialized with a random finite-amplitude perturbation superimposed on a laminar parabolic profile. The Reynolds number $Re = \frac{U h}{\nu}$ is varied between 2000 and 5000, where $U$ is the centerline velocity of the laminar profile, and $h$ is the half-channel height.

### Imposed Values
- Constant instantaneous volume flux in the $x$-direction:
$$ Q = \frac{1}{\lambda_z} \int_{0}^{\lambda_z} \int_{-h}^{h} u \, dy \, dz = \frac{4}{3} hU. $$
- Zero instantaneous averaged pressure gradient in the spanwise ($z$) direction:
$$ P(z=0) = P(z=\lambda_z). $$

## 2. The Computational Setup

### Grid and Resolution
The computational grid typically used was $32 \times 129 \times 16$ in the $x$, $y$ (normal to the walls), and $z$ directions, respectively, with the spanwise period of the box varied between 80 and 160 wall units, and the streamwise period between 300 and 600 wall units.

### Discretization of Space
A spectral method with dealiased Fourier expansions in the homogeneous directions and Chebyshev polynomials in the cross-channel coordinate is utilized for spatial discretization.

### CFD Method Used
A fully spectral Navier-Stokes solver with dealiased Fourier expansions for the streamwise and spanwise directions and Chebyshev polynomials for the normal direction.

### Time Integration Method and Parameters
Time was advanced using the second-order Adams-Bashforth scheme for the convective terms and the implicit second-order Crank-Nicolson scheme for the linear viscous terms. The maximum CFL number was maintained at approximately 0.2.


## 3. Post processing 

### Numerical Validation
Validation was performed through comparison with experimental data and theoretical predictions for low-order statistics such as mean velocity profiles and Reynolds stresses. The simulations reproduced several key features of near-wall turbulence accurately in the region below $y^+ = 40$.


### Diagnostic Quantities

#### Wall Shear Stress ($\tau_w$)
Wall shear stress is a critical diagnostic quantity that reflects the momentum transfer at the boundary. It can be defined as:

$$
\tau_w = \mu \left. \frac{\partial u}{\partial y} \right|_{y=0},
$$

where $\mu$ is the dynamic viscosity, $u$ is the streamwise velocity component, and $y=0$ denotes the wall location. The gradient $\frac{\partial u}{\partial y}$ is evaluated at the wall.

In the paper, the wall shears is define as follow (integrated over $x$ and $z$):
$$ \Omega(z=\pm 1) = \frac{1}{\lambda_x \lambda_z} \int_{0}^{\lambda_x} \int_{0}^{\lambda_z} \frac{\partial u}{\partial y} \bigg|_{\left(x, \pm 1, z\right)} \, dx \, dz.
 $$




#### Time-Averaged Wall Shears ($\overline{\tau_w}$)
The time-averaged wall shear stress provides insight into the mean flow characteristics over a period, defined as:

$$
\overline{\tau_w} = \frac{1}{T}\int_{0}^{T} \tau_w(t) \,dt,
$$

where $T$ is the averaging period.

In the paper, the time-averaged wall shears is written: 
$$\overline{\Omega}_{\tau}(z=\pm 1)$$

#### Local Friction Velocity ($u_{\tau}$)
The friction velocity is a scale for velocity derived from the wall shear stress, used to non-dimensionalize velocities in the turbulent boundary layer:

$$
u_{\tau} = \sqrt{\frac{\tau_w}{\rho}},
$$

where $\rho$ is the fluid density.
In the paper, the local friction velocities is written: 
$$u_{\tau}(z=\pm 1)=(\overline{\Omega}_{\tau}\nu)^{1/2}$$

#### Non-Dimensional Wall Units ($y^+$, $u^+$)
To analyze the flow in wall units, distances and velocities are non-dimensionalized using the viscous length scale and friction velocity:

$$
y^+ = \frac{y u_{\tau}}{\nu}, \quad u^+ = \frac{u}{u_{\tau}},
$$

where $y$ is the normal distance from the wall, and $u$ is the local flow velocity.

#### Turbulent Intensities ($u'^+$, $v'^+$, $w'^+$)
Turbulent intensities are statistical measures of the fluctuating components of velocity, defined as the square root of the velocity variances, non-dimensionalized by the friction velocity:

$$
u'^+ = \sqrt{\overline{{u'}^2}} / u_{\tau}, \quad v'^+ = \sqrt{\overline{{v'}^2}} / u_{\tau}, \quad w'^+ = \sqrt{\overline{{w'}^2}} / u_{\tau},
$$

where $u'$, $v'$, and $w'$ are the fluctuating components of velocity in the streamwise, normal, and spanwise directions, respectively, and the overbar denotes averaging in time.

#### Reynolds Stress ($-\overline{u'v'}^+$)
The Reynolds stress represents the turbulent momentum transfer, important for understanding the mechanics of turbulence production. It is non-dimensionalized by the square of the friction velocity:

$$
-\overline{u'v'}^+ = -\frac{\overline{u'v'}}{{u_{\tau}}^2},
$$

where $u'$ and $v'$ are the fluctuating components of velocity in the streamwise and normal directions, respectively.

These diagnostic quantities are essential for characterizing the state of the flow, its stability, and the mechanisms of turbulence production and maintenance within the channel flow simulation.


### Turbulent Analysis
The turbulent flow field exhibits a fundamental period of intermittency associated with the regeneration of turbulence. This process involves the wrapping of wall-layer vorticity around an inclined longitudinal vortex and is a key mechanism in sustaining turbulence within the simulated minimal flow unit.



***
## Questions
***

- Discuss about the difference between our Reynolds and theirs because 2000-5000 is high...

<div style="float:center; margin-left:100px;">
    <img src="Images/Literature/Jimenez_Reynolds.png" alt="Figure 1 du papier" width="800">
</div>
<div style="float:center; margin-left:100px;">
    <img src="Images/Literature/Jimenez_legend_fig4.png" alt="Figure 1 du papier" width="800">
</div>


# Analyse in details 

## Part 3 of the paper 
Part 3 of the paper "The minimal flow unit in near-wall turbulence" by Javier Jimenez and Parviz Moin focuses on the minimization of the computational box and the statistical properties of the flow within it. The authors explored various Reynolds numbers (2000, 3000, and 5000) to identify the smallest computational box that could sustain turbulence. This box is characterized by its ability to maintain non-laminar, stochastic, and three-dimensional flow.

### Key findings include:
- **Minimal Domain Scaling**: The critical channel span, when expressed in wall units ($\lambda_z^+$), is found to be roughly constant across different Reynolds numbers, with a critical value around 100. This aligns with experimental measurements of mean streak spacing in the viscous sublayer, suggesting a dynamical importance of this spacing.
- **Streamwise Length**: The minimal streamwise length required for sustaining turbulence is about 250-350 in wall units ($\lambda_x^+$), close to observed spacings of vortical structures in wall regions.
- **Sample Statistics**: The paper presents statistics of the minimal turbulent channel, including mean velocity profiles and Reynolds shear stress, showing good agreement with standard laws for turbulent channel flow in the near-wall region. 
- **One-sided Turbulent Channel Flow**: The authors observe and detail a state where turbulence exists only near one wall, with the flow near the other wall being nearly two-dimensional and less turbulent. This state, predominant at lower Reynolds numbers or minimal box spans, *showcases the independence of turbulence mechanisms on each wall*.
- **Wall Stresses**: They also discuss the agreement of their simulations with Dean's empirical correlation between wall skin friction and bulk velocity, further validating the minimal channel as a model for studying wall turbulence. *Figure 17, there is a link between $Re_{\tau}$ and $Re$.*

### Precisions 

<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/Jimenez_mean_streamwise_u.png" alt="Figure 7" width="600">
</div>
<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/Jimenez_shear_stress.png" alt="Figure 8" width="600">
</div>
<div style="float:right; margin-left:20px;">
    <img src="Images/Literature/Jimenez_RMS_u_v.png" alt="Figure 9" width="600">
</div>

#### Critical Span in "Minimal Domain Scaling"
The critical spanwise width ($\lambda_z^+$) for the computational box that can sustain turbulence is found to be around 100 wall units. This dimension aligns with experimental observations of mean streak spacing in the viscous sublayer. From a physical perspective, this critical span suggests that the dynamics of wall turbulence are intimately tied to the spacing between low-velocity streaks near the wall. It underscores the importance of this spatial scale in the self-sustaining process of wall turbulence, indicating that the minimal turbulent structures necessary for sustaining turbulence are of a size comparable to this critical span. The dynamical importance of this finding is that it bridges a gap between statistical observations in experiments and the underlying flow physics, highlighting the streak spacing not just as a recurrent pattern but as an essential feature of near-wall turbulence dynamics.

#### Precision on "Sample Statistics"
- **Mean Streamwise Velocity Profiles**: Figures 7 and 8 showcase the mean streamwise velocity profiles near the wall. Figure 7, specifically, presents velocity profiles for Reynolds numbers 3000 and 5000, comparing them against the standard logarithmic law of wall turbulence. It demonstrates that the flow in the minimal channel adheres well to known turbulent boundary layer behaviors in the near-wall region, particularly at lower Reynolds numbers.
- **Reynolds Shear Stress**: Figure 8 displays the normalized Reynolds shear stress profiles, illustrating how the flow achieves a statistically steady state. This figure indicates that the simulated turbulence captures essential features of the wall-bounded shear flows, consistent with experimental findings.
- **Root-Mean-Square Velocity Fluctuations**: Figure 9 compares the root-mean-square velocity fluctuations in the minimal channel with experimental values from Wei & Willmarth (1989). This comparison further validates the minimal channel model by showing a good match with experimental data, especially near the wall.

#### Details on "One-sided Turbulent Channel Flow"
The observation of one-sided turbulent channel flow represents a state where turbulence is maintained only near one wall, while the flow near the opposite wall remains nearly two-dimensional and less turbulent. This phenomenon is most pronounced at lower Reynolds numbers or when the spanwise width of the computational box approaches its minimal value. The transition to this state involves the following dynamics:
- **Initial Conditions**: The flow begins with turbulence potentially present on both walls or initiated from a state closer to turbulence on one wall.
- **Development**: Over time, turbulence decays on one of the walls, leading to a state where active turbulence is observed on only one side. This transition underscores the independence of turbulence mechanisms on each wall.
- **Intermittency and Regeneration**: Even in this one-sided turbulent state, the flow near the turbulent wall exhibits characteristics of intermittent turbulence, including bursts of activity that are crucial for the regeneration of turbulence.
- **Switching**: Remarkably, the turbulence can spontaneously switch from one wall to the other, a process that may involve complex interactions between the flow structures near each wall and the core flow. This switching further highlights the dynamic nature of near-wall turbulence and suggests a form of bistability in the flow, where two stable states (turbulence near either wall) are possible under similar flow conditions.

This one-sided turbulence phenomenon highlights the robustness of the turbulence regeneration mechanisms and suggests a significant degree of autonomy of the near-wall turbulence from the outer flow, especially in minimal domain configurations.


### Conclusions:
The minimal flow unit concept provides a significantly simplified model for studying the mechanics of wall-bounded flows and testing turbulence control strategies. This simplification retains the essential features of near-wall turbulence, particularly the low-order statistical quantities, within a minimal spatial domain. The findings suggest that the basic dynamics of wall turbulence are confined to a relatively small region, challenging previous understandings and highlighting the minimal channel as a promising "laboratory" for further investigations.

### Mathematical Formulation:
- The spanwise and streamwise lengths of the computational box are determined based on the requirements that the flow remains turbulent, stochastic, and three-dimensional.
- The critical spanwise width ($\lambda_z^+$) for sustaining turbulence is identified to be around 100 wall units, which correlates with the experimentally observed mean streak spacing.
- Streamwise length ($\lambda_x^+$) considerations point to a minimal period of about 250-350 wall units, indicating a lower bound for the length of eddies significant for near-wall turbulence dynamics.


## Part 4 of the paper
Part 4 of "The minimal flow unit in near-wall turbulence" deeply explores the turbulence regeneration cycle and the structure of the vorticity field within the minimal flow unit. This section is critical for understanding how turbulence sustains itself in near-wall conditions, especially within the confines of a minimal computational domain.

### Turbulence Regeneration Cycle:
- The intermittent nature of wall turbulence is highlighted, with turbulence characterized by quasi-periodic fluctuations in wall shear stress, indicating a fundamental regeneration cycle.
- A key finding is the identification of a very long timescale ($UT/h \approx 100$) associated with these fluctuations, suggesting that the structures involved are remarkably stable.
- This cycle involves periods where turbulence 'blooms,' affecting all flow statistics, followed by relatively quiescent phases where the flow regenerates its turbulent structures.

### Structure of the Vorticity Field:
- The paper presents a detailed analysis of the vorticity field structure in a minimal channel, emphasizing the arrangement of low-speed streaks, streamwise vortices, and shear layers.
- Each computational box contains a single set of wall structures: a low-speed streak, a detached shear layer, and streamwise vortices, illustrating a simplified yet comprehensive model of wall turbulence.
- The study illustrates how a compact streamwise vortex lifts the wall layer, creating a low-speed streak. This process is likened to the classical hairpin vortex model, albeit with significant asymmetry, suggesting a stable arrangement conducive to the long lifetimes of turbulent structures.

### Key Insights:
- The minimal channel effectively captures several statistical properties of near-wall turbulence, including the intermittent regeneration of turbulence and the structure of vorticity fields, despite its simplified periodic setup.
- The spanwise extent of the minimal computational domain scales with wall units ($\lambda_z^+ \approx 100$), aligning with experimentally observed streak spacings and underlining the dynamical significance of these spacings in sustaining wall turbulence.
- The paper suggests that the inner region of turbulent boundary layers can be modeled effectively with minimal domains, shedding light on the self-sustaining mechanisms of wall turbulence and offering a simplified framework for future turbulence studies.

### Conclusion:
Part 4 delves into the mechanisms underpinning the regeneration of turbulence and the intricate structure of the vorticity field within minimal flow units. These insights are pivotal for understanding the self-sustaining nature of wall turbulence and provide a foundation for simplified yet accurate modeling of turbulent boundary layers.


## Part 5 of the paper 
Part 5 of "The minimal flow unit in near-wall turbulence" delves into the dynamics and structure of the vorticity field within the minimal flow unit, particularly emphasizing the regeneration of turbulence and the intricate structure of the vorticity field.

### Dynamics of Vorticity Field:
- The study presents a comprehensive analysis of the vorticity field's structure in the minimal channel, focusing on the configuration and behavior of low-speed streaks, streamwise vortices, and shear layers.
- The minimal computational domain is revealed to contain essentially a single set of wall structures: a low-speed streak filled by a single, elongated streamwise vortex that significantly influences the flow dynamics by lifting a layer of spanwise vorticity away from the wall, creating the observed low-speed streak.

### Regeneration Mechanism:
- An in-depth examination of the regeneration mechanism of turbulence is provided, where the periodic wrapping of wall-layer vorticity around a single inclined longitudinal vortex is identified as a critical process. This mechanism is tied to the observed intermittency in wall shear stress, suggesting a fundamental period for the regeneration of turbulence.
- The study discusses the stability and evolution of these structures over time, noting their capacity to be convected downstream for extensive distances, which implies a high degree of stability.

### Structural Elements and Their Interactions:
- Detailed visualizations of the vorticity field showcase the complex interactions between the structural elements of wall turbulence. These include the arrangement of low-speed streaks, the role of streamwise vortices in generating these streaks, and the emergence of shear layers from the wall.
- The analysis explains how the streamwise vortices lift the spanwise vorticity layer, forming the low-speed streaks and detached shear layers. This process closely resembles the classical hairpin vortex model, albeit with significant asymmetry, suggesting a stable configuration that contributes to the long lifetimes of turbulent structures.

### Turbulence Regeneration Cycle:
- A very long timescale ($UT/h \approx 100$) is identified for the fluctuations in wall shear stress, indicating a fundamental regeneration cycle of turbulence. This cycle involves quasi-periodic 'blooms' of turbulence, affecting all flow statistics, followed by phases of relative quiescence where the flow regenerates its turbulent structures.
- The study highlights the independence of this regeneration cycle at each wall, further emphasizing the complex dynamics at play in sustaining near-wall turbulence.

### Implications and Conclusions:
- The findings from Part 5 offer profound insights into the self-sustaining mechanisms of wall turbulence, presenting a simplified yet accurate model for studying the fundamental dynamics of wall-bounded flows.
- The analysis underscores the minimal channel as an idealized laboratory for investigating turbulence mechanisms, suggesting that the essential dynamics of wall turbulence are confined to a relatively small spatial domain. This challenges previous understandings and highlights the minimal channel as a promising tool for future turbulence research and control strategies.

### Key Figures and Visualizations:
- Visualizations of the vorticity field reveal the detailed structure and dynamics of the turbulence regeneration cycle, including the interactions between low-speed streaks, streamwise vortices, and shear layers.
- The study utilizes three-dimensional views and time sequences of the flow field to illustrate the evolution of these structures, providing a vivid depiction of the turbulence regeneration process within the minimal flow unit.

In summary, Part 5 provides a detailed examination of the vorticity field and turbulence regeneration mechanisms within the minimal flow unit, offering valuable insights into the fundamental processes that sustain wall turbulence.
